In [1]:
cd /content/drive/MyDrive/Git/exercise/Code

/content/drive/MyDrive/Git/exercise/Code


# Module import 

In [2]:
import pandas as pd
import numpy as np
import random
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

# Prepare Data

## Define function

In [3]:
def bmi_calc(x):
    return np.round(x['ITEM_F002'] / ((x['ITEM_F001']/100)**2),1)

In [4]:
def age_categorize(x):
    if x.TEST_AGE < 10:
        return '10세 미만'
    elif x.TEST_AGE >= 70:
        return '70대 이상'
    else:
        return str((x.TEST_AGE // 10) * 10) + '대'

In [5]:
def bmi_categorize(x):
    bmi = bmi_calc(x)
    return '3단계비만' if bmi>=35 else ('2단계비만' if bmi>=30 else \
                                  ('1단계비만' if bmi>=25 else \
                                   ('비만전단계' if bmi>=23 else \
                                    ('정상체중' if bmi>=18.5 else '저체중'))))

## Load data

In [6]:
data = pd.read_csv('../Data/processed_data.csv')
user_group = pd.read_csv('../Data/user_group.csv')
sample_data = pd.read_csv('../Data/Sample_data.csv')

## Prepare sample user data

In [7]:
sample_data = sample_data.iloc[:300].rename(columns={'CERT_GBN':'CERT_GROUP'})
sample_data['ITEM_F018'] = sample_data.apply(bmi_calc, axis=1)
sample_data['AGE_GROUP'] = sample_data.apply(age_categorize,axis=1)
sample_data['BMI_GROUP'] = sample_data.apply(bmi_categorize,axis=1)
sample_data.head(5)

,TEST_AGE,CERT_GROUP,TEST_SEX,ITEM_F001,ITEM_F002,ITEM_F018,AGE_GROUP,BMI_GROUP
0,25,하,M,167.3,46.98,16.8,20대,저체중
1,19,하,M,179.0,84.00,26.2,10대,1단계비만
2,19,하,M,171.5,95.60,32.5,10대,2단계비만
3,20,하,F,158.4,43.84,17.5,20대,저체중
4,53,하,M,173.4,73.40,24.4,50대,비만전단계


In [8]:
group_sample_data = pd.merge(sample_data,user_group,how='left').reset_index().rename(columns={'index':'USER_ID'})
group_sample_data.head(5)

,USER_ID,TEST_AGE,CERT_GROUP,TEST_SEX,ITEM_F001,ITEM_F002,ITEM_F018,AGE_GROUP,BMI_GROUP,G_ID
0,0,25,하,M,167.3,46.98,16.8,20대,저체중,74
1,1,19,하,M,179.0,84.00,26.2,10대,1단계비만,56
2,2,19,하,M,171.5,95.60,32.5,10대,2단계비만,62
3,3,20,하,F,158.4,43.84,17.5,20대,저체중,77
4,4,53,하,M,173.4,73.40,24.4,50대,비만전단계,194


# Exercise Recommendation

## Count exercise set 

In [9]:
set_df = data[['G_ID','PRE_EX','MAIN_EX','FINISH_EX']].value_counts().reset_index().rename(columns={0:'COUNT'})
set_df[set_df.G_ID==92].head(5)

,G_ID,PRE_EX,MAIN_EX,FINISH_EX,COUNT
0,92,걷기,"달리기, 무릎 높여 제자리 달리기",전신 루틴 스트레칭,422
4,92,유산소 운동 전 동적 루틴 스트레칭,"윗몸올리기 , 누워서 엉덩이 들어올리기, 엎드려 버티기, 서서 상체 일으키기, 앉아...",전신 루틴 스트레칭,250
40,92,전신 루틴 스트레칭,"앉아서 당겨 내리기, 앉아서 뒤로 당기기, 앉아서 밀기, 앉아서 모으기, 한발 앞으...",전신 루틴 스트레칭,134
50,92,"깍지 끼고 상체 숙이기, 목 스트레칭, 등/어깨 뒤쪽 스트레칭, 어깨 뒤쪽 스트레칭...","몸통 들어올리기, 소파를 이용한 윗몸올리기, 엎드려 버티기, 옆으로 누워 버티기, ...","깍지 끼고 상체 숙이기, 목 스트레칭, 등/어깨 뒤쪽 스트레칭, 아래 팔 스트레칭,...",120
57,92,"하지 루틴 스트레칭, 상지 루틴 스트레칭, 유산소 운동 전 동적 루틴 스트레칭","윗몸올리기 , 엎드려 버티기, 옆으로 누워 버티기, 누워서 엉덩이 들어올리기, 한발...","전신 루틴 스트레칭, 자가근막이완술 루틴 스트레칭, 하지 루틴 스트레칭, 등/어깨 ...",109


## Define function

In [10]:
# 운동 리스트를 통해 희소행렬 도출
# input : exercise list / output : sparse matrix
def sparse_matrix(ex_list):
  te = TransactionEncoder()
  te_ary = te.fit(ex_list).transform(ex_list)
  sparse_matrix = pd.DataFrame(te_ary, columns=te.columns_)
  return sparse_matrix

In [11]:
# 연관분석을 통해 선택된 운동과 연관성이 높은 운동 3개를 리턴
def ranking(pre_df, main_df, finish_df, ex_list):
  global df_list
  df_list = [pre_df, main_df, finish_df] # 단계별 연관분석 결과값
  result_list = [] # 결과값을 리스트에 담아 하나씩 출력
  result_df = pd.DataFrame(columns=pre_df.columns) # 빈 DataFrame 생성

  for i in range(len(df_list)): # 운동 단계별 for문 pre, main, finish
    global tmp_df 
    tmp_df = df_list[i] 
    for each_ex in ex_list[i].split(','): # 각 단계별 운동들을 리스트로 분할
      each_ex = each_ex # 운동이름에 공백 제거
      if tmp_df.empty:
        result_df = pd.DataFrame(columns=pre_df.columns)
      else:
        each_result = tmp_df[tmp_df['antecedents']==frozenset({each_ex})].sort_values('support') # 추천 세트 운동과의 연관분석 결과를 지지도 순으로 정렬
        each_result['consequents'] = each_result['consequents'].apply(lambda x : list(x)[0]) # frozenset to string, 타입변환
        length = 3 if len(each_result) else len(each_result) # set length
        result_df = pd.concat([result_df,each_result.iloc[:length]]) # 연관성이 높은 운동이 3개 이상일 경우 3개만 저장
    result_list.append(result_df['consequents']) # 결과값 리스트에 추가
    result_df = None # 초기화
  return list(result_list[0]), list(result_list[1]), list(result_list[2]) # 준비, 본, 마무리 운동 연관성 높은 운동 리턴

In [12]:
# select main routine and additional routine
def select_exercise(pre_list, main_list, finish_list, max):
  main_reco_list = [] # 메인 추천 리스트
  sub_reco_list = [] # 추가 추천 리스트
  tmp_list = []
  asso_list = [pre_list,main_list,finish_list] # 연관 있는 운동 리스트
  tmp_set = set()
  random_num = random.randint(1, 100) # 임의의 1개 운동을 선택하기 위한 난수 생성
  
  for i, str in enumerate(max):
    ex_list = str.split(',') # 운동 세트에서 각 단계별 운동들
    reco_ex = ex_list[random_num %len(ex_list)] # 난수를 통해 임의의 1개 운동 선택
    main_reco_list.append(reco_ex) # 메인 추천 리스트에 선택된 운동 삽입

    tmp_set.update(ex_list) # 임시 리스트에 max 운동 리스트 삽입
    tmp_set.remove(reco_ex) # 선택된 운동 삭제 -> 단계별 최빈 세트 운동에서 메인 추천 운동 제거
    tmp_set.update(asso_list[i]) # 연관 분석을 통해 추천된 운동들 추가
    tmp_list.append(list(tmp_set)) # 해당 set을 리스트로 변환하여 tmp_list에 추가 -> [ [준,], [본,], [마,] ]
  for i, reco in enumerate(tmp_list):
    if (len(reco)) == 0:
      random_list = ['없음']
    elif (len(reco)) < 4:
      random_list = reco # 운동 개수가 4개 이하면 전체 운동 적용
    else:
      random_list = random.sample(reco, 4) # 운동 개수가 4개 초과일 경우 임의로 4개 선택
    sub_reco_list.append(random_list)
  return list(main_reco_list), list(sub_reco_list)

In [13]:
# select top 1 exercise set from user info
# input : group_id / ouput : recommend_set, sub_recommend_set
def mode_set(g_id, min_support = 0.01, min_threshold=0.2):
  user_set = set_df[set_df['G_ID']==g_id] # 유저가 해당하는 그룹의 처방 운동 세트 빈도
  max_ex_set = list(user_set[user_set['COUNT'] == user_set['COUNT'].max()].iloc[0,1:4]) # 가장 많이 처방된 운동 세트

  # 해당 그룹의 단계별 운동 리스트 생성
  pre_ex_all = list(user_set['PRE_EX'].apply(lambda x : x.split(','))) 
  main_ex_all = list(user_set['MAIN_EX'].apply(lambda x : x.split(',')))
  finish_ex_all = list(user_set['FINISH_EX'].apply(lambda x : x.split(',')))

  # 단계별 sparse matrix 생성
  pre_sparse = sparse_matrix(pre_ex_all)
  main_sparse = sparse_matrix(main_ex_all)
  finish_sparse = sparse_matrix(finish_ex_all)

  # 지지도가 0.01 이상인 운동 선택
  pre_itemsets = apriori(pre_sparse, min_support = min_support, use_colnames=True,max_len=2)
  main_itemsets = apriori(main_sparse, min_support = min_support, use_colnames=True,max_len=2)
  finish_itemsets = apriori(finish_sparse, min_support = min_support, use_colnames=True,max_len=2)

  # 그 중에서 신뢰도가 0.2 이상인 운동 선택
  pre_select = association_rules(pre_itemsets, metric="confidence", min_threshold=min_threshold)
  main_select = association_rules(main_itemsets, metric="confidence", min_threshold=min_threshold)
  finish_select = association_rules(finish_itemsets, metric="confidence", min_threshold=min_threshold)

  # consequents의 길이가 1인 것만 선택 -> 1개의 운동끼리 비교해서 추천하도록
  pre_select = pre_select[pre_select['consequents'].apply(lambda x : len(x) == 1)]
  main_select = main_select[main_select['consequents'].apply(lambda x : len(x) == 1)] 
  finish_select = finish_select[finish_select['consequents'].apply(lambda x : len(x) == 1)]
  
  # ranking 
  pre_rank, main_rank, finish_rank = ranking(pre_select, main_select, finish_select, max_ex_set)

  # select exercise 
  recommend_set, sub_recommend_set = select_exercise(pre_rank, main_rank, finish_rank, max_ex_set)
  return recommend_set, sub_recommend_set

## Recommendation

In [14]:
main_reco = []
sub_reco = []
u_id_list = []

In [15]:
for u_id, g_id in group_sample_data[['USER_ID','G_ID']].values:
    best, sub = mode_set(g_id) # 추천 루틴 및 추가 추천 운동
    main_reco.append(list(best)) 
    sub_reco.append(sub)
    u_id_list.append(u_id)

In [16]:
group_sample_data.head(3)

,USER_ID,TEST_AGE,CERT_GROUP,TEST_SEX,ITEM_F001,ITEM_F002,ITEM_F018,AGE_GROUP,BMI_GROUP,G_ID
0,0,25,하,M,167.3,46.98,16.8,20대,저체중,74
1,1,19,하,M,179.0,84.00,26.2,10대,1단계비만,56
2,2,19,하,M,171.5,95.60,32.5,10대,2단계비만,62


In [17]:
# DataFrame 생성
reco_df = pd.DataFrame(main_reco,index=u_id_list,columns=['Recommend_pre_ex','Recommend_main_ex','Recommend_finish_ex'])
sub_df = pd.DataFrame(sub_reco,index=u_id_list,columns=['Additional_pre_ex','Additional_main_ex','Additional_finish_ex'])
merge_df = pd.merge(reco_df,sub_df,left_index=True,right_index=True).reset_index().rename(columns={'index':'USER_ID'})
result_df = pd.merge(group_sample_data,merge_df)
for column in result_df.iloc[:,-3:]:
  result_df[column] = result_df[column].apply(lambda x : (', ').join(x))

In [18]:
result_df.head(3)

,USER_ID,TEST_AGE,CERT_GROUP,TEST_SEX,ITEM_F001,ITEM_F002,ITEM_F018,AGE_GROUP,BMI_GROUP,G_ID,Recommend_pre_ex,Recommend_main_ex,Recommend_finish_ex,Additional_pre_ex,Additional_main_ex,Additional_finish_ex
0,0,25,하,M,167.3,46.98,16.8,20대,저체중,74,전신 루틴 스트레칭,누워서 다리 들어올리기,전신 루틴 스트레칭,유산소 운동 전 동적 루틴 스트레칭,"앉았다 일어서기, 엎드려서 팔 다리 들기, 윗몸올리기 , 누워서 엉덩이 들어올리기","계단 올라갔다 내려오기, 한발 앞으로 내밀고 앉았다 일어서기, 뒤꿈치 들기, ..."
1,1,19,하,M,179.0,84.00,26.2,10대,1단계비만,56,유산소 운동 전 동적 루틴 스트레칭,걷기,전신 루틴 스트레칭,없음,"달리기, 앉아서 위로 밀기, 앉아서 다리 굽히기, 자전거타기","서서 상체 일으키기, 팔굽혀펴기, 달리기, 몸통 들어올리기"
2,2,19,하,M,171.5,95.60,32.5,10대,2단계비만,62,걷기,무릎 높여 제자리 달리기,전신 루틴 스트레칭,없음,"엎드려 버티기, 수영, 계단 올라갔다 내려오기, 줄넘기","계단 올라갔다 내려오기, 줄넘기, 엎드려 버티기, 스텝퍼 뛰어서 오르내리기"


In [19]:
result_df.to_csv('../Data/sample_recommend_result.csv',encoding='utf-8',index=False)

# Trial sample

In [20]:
age = int(input('나이 : '))
cert = input('운동강도(상/중/하) :')
sex = input('성별(F/M) :')
height = np.float64(round(float(input('키 :')),1))
weight = np.float64(round(float(input('몸무게 :')),1))

나이 : 27
운동강도(상/중/하) :중
성별(F/M) :F
키 :164.6
몸무게 :55.2


In [21]:
input_list = [22222,age,'하',sex,height,weight]
input_df = pd.DataFrame(input_list,index=['USER_ID', 'TEST_AGE','CERT_GROUP','TEST_SEX', 'ITEM_F001', 'ITEM_F002']).T.astype({'USER_ID':'int', 'TEST_AGE':'int', 'ITEM_F001':'float', 'ITEM_F002':'float'})
input_df['ITEM_F018'] = input_df.apply(bmi_calc, axis=1)
input_df['AGE_GROUP'] = input_df.apply(age_categorize,axis=1)
input_df['BMI_GROUP'] = input_df.apply(bmi_categorize,axis=1)
input_df = pd.merge(input_df,user_group,how='left').reset_index().rename(columns={'index':'USER_ID'})
pd.DataFrame(mode_set(int(input_df.G_ID)),columns=['준비운동','본운동','마무리운동'],index=['추천 운동','추가 추천 운동']).T

,추천 운동,추가 추천 운동
준비운동,걷기,[없음]
본운동,달리기,"[ 사다리 옆으로 발 옮기기, 걷기, 달리기, 수영]"
마무리운동,전신 루틴 스트레칭,"[ 달리기, 사다리 옆으로 발 옮기기, 걷기, 무릎 높여 제자리 달리기]"
